---
title: Graphs
order: 5
---

This post implements (and improves) code in chapter 14 from the textbook Introduction to Computation and Programming Using Python, With Application to Computational Modeling and Understanding Data, third edition, by John V. Guttag, 2020. 

In [1]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Tuple

In [2]:
@dataclass(frozen=True)
class Node:
    """Basic immutable graph node."""

    name: str

    def __str__(self) -> str:
        return self.name

Designing the inheritance pattern is difficult to me here. The substitution principle: A code that works on a superclass must work on a subclass. But if a code works on bidirectional edges (perhaps exploiting symmetry) it may not work on unidirectional edges (where the symmetry doesn't exist). So, if anything, bidirectional edge must inherit from unidirectional edge. But that doesn't work either, because unidirectional edge logically must have the sources and destinations, but bidirectional edge cannot have them. Which node should be returned by a bidirectional edge's "source" property? There is no clear answer, so I conclude bidirectional edge cannot inherit from unidirectional. Together these two facts mean neither can inherit from the other.

One solution would be to completely discard bidirectional edges and only work with directional ones - after all, we can build undirected graph with directed edges. But that would mean, while building an undirected graph, that we treat unidirectional edges as bidirectional, which is weird.

BaseEdge is a abstract base class for edges. An edge has two nodes, node1 and node2, can return them in a tuple (.nodes) and can be represented by a string where names of both nodes are separated by a separator. Also is hashable, which means it implements \_\_eq\_\_ and \_\_hash\_\_ methods.
_separator, \_\_eq\_\_ and \_\_hash\_\_ are abstract methods. Their implementation depends on whether the edge is bidirectional or unidirectional.

@ property means accessing `.nodes` on in instance will call this *method*, which works as a getter. It also blocks setting the attribute (via `.nodes = something`). But it doesn't block its mutation, were it e.g. a list.

In [3]:
@dataclass(frozen=True)
class BaseEdge(ABC):
    """Abstract base class for graph edge."""

    node1: Node
    node2: Node

    @property
    def nodes(self) -> Tuple[Node, Node]:
        return self.node1, self.node2

    @property
    @abstractmethod
    def _separator(self) -> str:
        pass

    def __str__(self) -> str:
        node1, node2 = self.nodes
        return f"{node1} {self._separator} {node2}"


@dataclass(frozen=True)
class Edge(BaseEdge):
    """Bidirectional unweighted graph edge."""

    def __post_init__(self):
        # Canonical ordering
        if self.node1.name > self.node2.name:
            node1 = self.node1
            object.__setattr__(self, "node1", self.node2)
            object.__setattr__(self, "node2", node1)

    @property
    def _separator(self) -> str:
        return "<---->"


@dataclass(frozen=True)
class DirectedEdge(BaseEdge):
    """Unidirectional unweighted graph edge."""

    def __init__(self, src: Node, dest: Node):
        # need to change the signature of the contructor
        # maintains the invariant that src=node1 and dest=node2
        BaseEdge.__init__(self, src, dest)

    @property
    def src(self) -> Node:
        return self.node1

    @property
    def dest(self) -> Node:
        return self.node2

    @property
    def _separator(self) -> str:
        return ">---->"


@dataclass(frozen=True)
class WeightedEdge(Edge):
    """Weighted bidirectional graph edge."""

    weight: int | float

    @property
    def _separator(self) -> str:
        return f"<--({self.weight})-->"


@dataclass(frozen=True)
class WeightedDirectedEdge(DirectedEdge):
    """Weighted unidirectional graph edge."""

    weight: int | float

    @property
    def _separator(self) -> str:
        return f">--({self.weight})-->"

In [4]:
# BaseEdge()  # TypeError: Can't instantiate abstract class BaseEdge without an implementation for abstract method '_separator'
node1 = Node("1")
node2 = Node("2")

print(node1 == node2)

False


In [5]:
edge12 = Edge(node1, node2)
w_edge12_314 = WeightedEdge(node1, node2, 3.14)
dir_edge12 = DirectedEdge(node1, node2)
wdir_edge12_314 = WeightedDirectedEdge(node1, node2, 3.14)

In [6]:
edge21 = Edge(node2, node1)
w_edge21_314 = WeightedEdge(node2, node1, 3.14)
dir_edge21 = DirectedEdge(node2, node1)
wdir_edge21_314 = WeightedDirectedEdge(node2, node1, 3.14)

In [7]:
w_edge12_3 = WeightedEdge(node1, node2, 3)
wdir_edge12_3 = WeightedDirectedEdge(node1, node2, 3)

In [8]:
print(f"{edge12} equals {edge21} is {edge12 == edge21}")
print(f"{w_edge12_314} equals {w_edge21_314} is {w_edge12_314 == w_edge21_314}")
print(f"{dir_edge12} equals {dir_edge21} is {dir_edge12 == dir_edge21}")
print(
    f"{wdir_edge12_314} equals {wdir_edge21_314} is {wdir_edge12_314 == wdir_edge21_314}"
)
print(f"{w_edge12_314} equals {w_edge12_3} is {w_edge12_314 == w_edge12_3}")
print(f"{wdir_edge12_314} equals {wdir_edge12_3} is {wdir_edge12_314 == wdir_edge12_3}")

1 <----> 2 equals 1 <----> 2 is True
1 <--(3.14)--> 2 equals 1 <--(3.14)--> 2 is True
1 >----> 2 equals 2 >----> 1 is False
1 >--(3.14)--> 2 equals 2 >--(3.14)--> 1 is False
1 <--(3.14)--> 2 equals 1 <--(3)--> 2 is False
1 >--(3.14)--> 2 equals 1 >--(3)--> 2 is False


In [9]:
print(edge12 is edge21)
print({edge12, edge21})  # exactly what we wanted because they are the same edge!

False
{Edge(node1=Node(name='1'), node2=Node(name='2'))}


In [10]:
{w_edge12_314, w_edge21_314}

{WeightedEdge(node1=Node(name='1'), node2=Node(name='2'), weight=3.14)}

In [11]:
class C:
    pass


c1 = C()
c2 = C()

{c1, c2}

{<__main__.C at 0x2af26b3e710>, <__main__.C at 0x2af26b69be0>}